In [1]:
import numpy as np
import pandas as pd

In [2]:
corpus_df = pd.read_csv("../incomplete_corpus_data/PotsdamSearchCorpus.dat", sep = "\t")
#testing with raw data from participant 1
raw_data = pd.read_csv("../incomplete_corpus_data/asc\SFC1_2.dat", sep = "\t", 
                       header = None, names=['time', 'x', 'y', 'pupil'])

In [3]:
#then I have to filter in corpus_df which data is for participant 1, but later I should optimize
#such that I import data and read from the corpus for all participants
corpus_p1 = corpus_df[corpus_df.subject == 1]

In [4]:
#USE THIS TO WRITE DATA IN ANOTHER FILES
"""
output_df = pd.DataFrame({'column_name': column_values})  
output_df.to_csv('output_file_name.dat')
"""

"\noutput_df = pd.DataFrame({'column_name': column_values})  \noutput_df.to_csv('output_file_name.dat')\n"

In [5]:
def get_valid_times(raw_data, corpus_data):
    """
    get all valid time stamps, i.e, filter the times such 
    that all times in the raw data are in between the trialstart and trialend times
    in the corpus_data.
    Here corpus_data corresponds to the data already filtered by participant number to match 
    the participant number from the raw_data file.
    :param raw_data: pandas data frame with columns = =['time', 'x', 'y', 'pupil']
    :param corpus_data: pandas data frame filtered for participant number #n
    
    :return: raw_data with only valid time stamps
    """
    
    n_trials = corpus_data['trialno'].iloc[-1] #get number of trials
    intervals = np.zeros((n_trials,2))
    for trial in range(n_trials):
        intervals[trial,0]=corpus_data[corpus_data['trialno']==trial+1]["trialstart"].iloc[0]
        intervals[trial,1]=corpus_data[corpus_data['trialno']==trial+1]["trialend"].iloc[0]
        
    ##subtract time zero 
    time_zero = corpus_data["trialstart"].iloc[0]
    raw_data['time'] -= time_zero
    raw_data = raw_data.drop(raw_data[raw_data.time < 0].index)
    raw_data['time'] += time_zero
    
    for trial in range(1,n_trials):
        end_last = intervals[trial-1,1] #end of the last trial
        beg_curr = intervals[trial,0] #beggining of the current trial
        raw_data = raw_data.drop(raw_data[(raw_data.time > end_last) & (raw_data.time < beg_curr)].index)

    return raw_data

In [6]:
filtered_raw = get_valid_times(raw_data, corpus_p1)